## Make pickle
- bbox 좌표와 배경을 포함한 레이블을 담고 있습니다

### ready to load Data

In [1]:
import glob as glob
import pickle
import cv2
import xml.etree.ElementTree as ET
from tqdm import tqdm_notebook as tqdm
import numpy as np

VOC_CLASSES=['background','aeroplane', 'bicycle', 'bird', 'boat',
             'bottle', 'bus', 'car', 'cat',
             'chair', 'cow', 'diningtable', 'dog',
             'horse', 'motorbike', 'person', 'pottedplant',
             'sheep', 'sofa', 'train', 'tvmonitor']

# train_val 데이터만 작성합니다.
DATA_PATH = './train/'

annotation_path = DATA_PATH + 'Annotations/'
image_path = DATA_PATH + 'JPEGImages/'

all_images = glob.glob(image_path + '*.jpg')
all_xmls = glob.glob(annotation_path + '*.xml')
print(len(all_images), len(all_xmls))

### Make ground_truth

In [70]:
def bbox_basic_parsing(coords, w, h):
    coords[0] = coords[0]/w
    coords[1] = coords[1]/h
    coords[2] = coords[2]/w
    coords[3] = coords[3]/h
    
    return coords

gt = dict()

for image_path, xml_path in tqdm(zip(all_images, all_xmls)):
    image = cv2.imread(image_path)
    image_name = image_path.split('\\')[-1]
    h, w, _ = image.shape
    
    sub_gt = []
    
    tree = ET.parse(xml_path)
    root = tree.getroot()
    
    if len(root.findall('object')) > 0:
        for obj in root.findall('object'):
            name = obj.find('name').text
            label = VOC_CLASSES.index(name)
            dummy_label = [0] * len(VOC_CLASSES)
            dummy_label[label] = 1
            xmin = int(obj.find('bndbox').find('xmin').text)
            ymin = int(obj.find('bndbox').find('ymin').text)
            xmax = int(obj.find('bndbox').find('xmax').text)
            ymax = int(obj.find('bndbox').find('ymax').text)
            
            xmin, ymin, xmax, ymax = bbox_basic_parsing([xmin, ymin, xmax, ymax], w, h)
            
            sub_gt.append(np.array([xmin, ymin, xmax, ymax] + dummy_label))
            
    gt[image_name] = np.array(sub_gt)

<ipython-input-70-267aa93f66fa>:11: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for image_path, xml_path in tqdm(zip(all_images, all_xmls)):


In [72]:
print(len(gt))

5011


In [73]:
print(gt['000005.jpg'])

[[0.526      0.56266667 0.648      0.904      0.         0.
  0.         0.         0.         0.         0.         0.
  0.         1.         0.         0.         0.         0.
  0.         0.         0.         0.         0.         0.
  0.        ]
 [0.33       0.704      0.506      0.992      0.         0.
  0.         0.         0.         0.         0.         0.
  0.         1.         0.         0.         0.         0.
  0.         0.         0.         0.         0.         0.
  0.        ]
 [0.01       0.65066667 0.134      0.99733333 0.         0.
  0.         0.         0.         0.         0.         0.
  0.         1.         0.         0.         0.         0.
  0.         0.         0.         0.         0.         0.
  0.        ]
 [0.482      0.51733333 0.59       0.79733333 0.         0.
  0.         0.         0.         0.         0.         0.
  0.         1.         0.         0.         0.         0.
  0.         0.         0.         0.         0.         0

### Make pickle!

In [76]:
with open('VOC2007.pkl', 'wb') as handle:
    pickle.dump(gt, handle)